# Project 1 - Prediction of Instagram Followers

Creation of a conda environment an adding it to Jupyter Notebook.
<div class="alert alert-block alert-success" style="font-family: monospace">
conda create --name ml<br>
conda activate ml<br>
conda install -c anaconda ipykernel<br>
python -m ipykernel install --user --name=ml<br>
</div>

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf

# Data

In [ ]:
# 01_Instagram_Followers_Prediciton